# 📊 Módulo 2: Agregaciones y GROUP BY en Banking

## 🎯 **Objetivos de Aprendizaje**

En este módulo aprenderás a:
- 📈 **GROUP BY**: Agrupar datos bancarios por segmentos
- 🧮 **Funciones de agregación**: SUM, AVG, COUNT, MAX, MIN
- 📊 **KPIs básicos**: Análisis de cartera, saldos, clientes
- 🔍 **HAVING**: Filtros avanzados en grupos agregados
- 💼 **Casos reales**: Reportes bancarios fundamentales

---

## 🏦 **Contexto Bancario**

**¿Por qué son importantes las agregaciones en Banking?**

- 💰 **Análisis de cartera**: Saldos totales por producto
- 📍 **Performance por sucursal**: Indicadores regionales
- 👥 **Segmentación de clientes**: Comportamiento por grupos
- 📈 **KPIs regulatorios**: Métricas de supervisión bancaria
- 🎯 **Toma de decisiones**: Data-driven insights

## 🔧 **1. Configuración del Entorno**

In [ ]:
# Importar librerías necesarias
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Librerías importadas exitosamente")
print(f"📅 Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Conectar a la base de datos bancaria
def conectar_bd():
    """
    Establece conexión con la base de datos bancaria
    """
    try:
        conn = sqlite3.connect('../../data/banking_core.db')
        print("✅ Conexión a banking_core.db establecida")
        return conn
    except Exception as e:
        print(f"❌ Error al conectar: {e}")
        return None

# Establecer conexión
conn = conectar_bd()

## 📊 **2. Explorando la Estructura de Datos**

In [ ]:
# Revisar las tablas disponibles
query_tablas = """
SELECT name 
FROM sqlite_master 
WHERE type='table'
ORDER BY name;
"""

tablas = pd.read_sql_query(query_tablas, conn)
print("📋 Tablas disponibles en banking_core.db:")
print(tablas.to_string(index=False))

In [ ]:
# Vista rápida de las tablas principales
def explorar_tabla(tabla, limit=5):
    """
    Muestra una vista previa de cualquier tabla
    """
    query = f"SELECT * FROM {tabla} LIMIT {limit};"
    df = pd.read_sql_query(query, conn)
    print(f"\n🔍 Vista previa de {tabla}:")
    print(df.to_string(index=False))
    print(f"📊 Columnas: {list(df.columns)}")
    return df

# Explorar las tablas principales
explorar_tabla('clientes', 3)
explorar_tabla('cuentas', 3)
explorar_tabla('productos_financieros', 3)

## 🧮 **3. Funciones de Agregación Básicas**

### **Las 5 funciones esenciales:**
- 🔢 **COUNT()**: Contar registros
- ➕ **SUM()**: Sumar valores
- 📊 **AVG()**: Calcular promedio
- ⬆️ **MAX()**: Valor máximo
- ⬇️ **MIN()**: Valor mínimo

In [ ]:
# COUNT: Contar registros
print("🔢 FUNCIÓN COUNT - Contando registros")
print("=" * 50)

# Contar total de clientes
query_count = "SELECT COUNT(*) as total_clientes FROM clientes;"
resultado = pd.read_sql_query(query_count, conn)
print(f"Total de clientes en el banco: {resultado['total_clientes'].iloc[0]:,}")

# Contar clientes activos
query_activos = "SELECT COUNT(*) as clientes_activos FROM clientes WHERE estado = 'ACTIVO';"
resultado_activos = pd.read_sql_query(query_activos, conn)
print(f"Clientes activos: {resultado_activos['clientes_activos'].iloc[0]:,}")

# Contar cuentas por estado
query_cuentas = """
SELECT estado, COUNT(*) as cantidad_cuentas
FROM cuentas 
GROUP BY estado;
"""
cuentas_estado = pd.read_sql_query(query_cuentas, conn)
print("\n📊 Cuentas por estado:")
print(cuentas_estado.to_string(index=False))

In [ ]:
# SUM: Sumar valores monetarios
print("➕ FUNCIÓN SUM - Sumando valores")
print("=" * 50)

# Saldo total en todas las cuentas
query_saldo_total = "SELECT SUM(saldo_actual) as saldo_total_banco FROM cuentas;"
saldo_total = pd.read_sql_query(query_saldo_total, conn)
total = saldo_total['saldo_total_banco'].iloc[0]
print(f"💰 Saldo total del banco: ${total:,.2f}")

# Saldo total por tipo de cuenta
query_saldo_tipo = """
SELECT 
    pf.categoria,
    SUM(c.saldo_actual) as saldo_total,
    COUNT(*) as numero_cuentas
FROM cuentas c
JOIN productos_financieros pf ON c.producto_id = pf.producto_id
GROUP BY pf.categoria
ORDER BY saldo_total DESC;
"""
saldo_por_tipo = pd.read_sql_query(query_saldo_tipo, conn)
print("\n📊 Saldo total por tipo de producto:")
print(saldo_por_tipo.to_string(index=False))

In [ ]:
# AVG: Calcular promedios
print("📊 FUNCIÓN AVG - Calculando promedios")
print("=" * 50)

# Saldo promedio por cuenta
query_promedio = "SELECT AVG(saldo_actual) as saldo_promedio FROM cuentas;"
promedio = pd.read_sql_query(query_promedio, conn)
avg_saldo = promedio['saldo_promedio'].iloc[0]
print(f"💵 Saldo promedio por cuenta: ${avg_saldo:,.2f}")

# Promedio de ingresos por segmento de cliente
query_ingresos_segmento = """
SELECT 
    segmento_cliente,
    AVG(ingresos_mensuales) as ingreso_promedio,
    COUNT(*) as cantidad_clientes
FROM clientes 
WHERE estado = 'ACTIVO'
GROUP BY segmento_cliente
ORDER BY ingreso_promedio DESC;
"""
ingresos_segmento = pd.read_sql_query(query_ingresos_segmento, conn)
print("\n📈 Ingresos promedio por segmento:")
print(ingresos_segmento.to_string(index=False))

In [ ]:
# MAX y MIN: Valores extremos
print("⬆️⬇️ FUNCIONES MAX y MIN - Valores extremos")
print("=" * 50)

# Saldos máximo y mínimo
query_extremos = """
SELECT 
    MAX(saldo_actual) as saldo_maximo,
    MIN(saldo_actual) as saldo_minimo,
    MAX(saldo_actual) - MIN(saldo_actual) as rango_saldos
FROM cuentas;
"""
extremos = pd.read_sql_query(query_extremos, conn)
print("💰 Análisis de saldos extremos:")
print(extremos.to_string(index=False))

# Cliente con mayores y menores ingresos por segmento
query_ingresos_extremos = """
SELECT 
    segmento_cliente,
    MAX(ingresos_mensuales) as ingreso_maximo,
    MIN(ingresos_mensuales) as ingreso_minimo
FROM clientes 
WHERE estado = 'ACTIVO'
GROUP BY segmento_cliente
ORDER BY ingreso_maximo DESC;
"""
ingresos_extremos = pd.read_sql_query(query_ingresos_extremos, conn)
print("\n📊 Ingresos extremos por segmento:")
print(ingresos_extremos.to_string(index=False))

## 📈 **4. GROUP BY - Agrupación de Datos**

### **¿Qué es GROUP BY?**
GROUP BY nos permite **agrupar registros** que tienen valores similares en una o más columnas, y luego aplicar funciones de agregación a cada grupo.

**Sintaxis básica:**
```sql
SELECT columna, FUNCION_AGREGACION()
FROM tabla
GROUP BY columna;
```

In [ ]:
# GROUP BY básico: Agrupación por una columna
print("📊 GROUP BY BÁSICO - Agrupación simple")
print("=" * 50)

# Número de clientes por segmento
query_segmento = """
SELECT 
    segmento_cliente,
    COUNT(*) as cantidad_clientes,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM clientes), 2) as porcentaje
FROM clientes
GROUP BY segmento_cliente
ORDER BY cantidad_clientes DESC;
"""
segmentacion = pd.read_sql_query(query_segmento, conn)
print("👥 Distribución de clientes por segmento:")
print(segmentacion.to_string(index=False))

# Visualizar la distribución
plt.figure(figsize=(10, 6))
plt.pie(segmentacion['cantidad_clientes'], 
        labels=segmentacion['segmento_cliente'], 
        autopct='%1.1f%%',
        startangle=90)
plt.title('Distribución de Clientes por Segmento')
plt.axis('equal')
plt.show()

In [ ]:
# GROUP BY con múltiples columnas
print("📊 GROUP BY MÚLTIPLE - Agrupación por varias columnas")
print("=" * 60)

# Análisis por segmento y estado
query_multi = """
SELECT 
    segmento_cliente,
    estado,
    COUNT(*) as cantidad,
    AVG(ingresos_mensuales) as ingreso_promedio
FROM clientes
GROUP BY segmento_cliente, estado
ORDER BY segmento_cliente, estado;
"""
analisis_multi = pd.read_sql_query(query_multi, conn)
print("📈 Análisis por segmento y estado:")
print(analisis_multi.to_string(index=False))

# Crear tabla pivote para mejor visualización
pivot_segmento = analisis_multi.pivot_table(
    index='segmento_cliente', 
    columns='estado', 
    values='cantidad', 
    fill_value=0
)
print("\n📋 Tabla pivote - Clientes por segmento y estado:")
print(pivot_segmento)

In [ ]:
# GROUP BY con JOINs - Análisis avanzado
print("🔗 GROUP BY + JOIN - Análisis integral")
print("=" * 50)

# Análisis de productos financieros
query_productos = """
SELECT 
    pf.categoria,
    pf.nombre_producto,
    COUNT(c.cuenta_id) as numero_cuentas,
    SUM(c.saldo_actual) as saldo_total,
    AVG(c.saldo_actual) as saldo_promedio,
    MAX(c.saldo_actual) as saldo_maximo
FROM productos_financieros pf
LEFT JOIN cuentas c ON pf.producto_id = c.producto_id
GROUP BY pf.categoria, pf.nombre_producto
ORDER BY saldo_total DESC;
"""
analisis_productos = pd.read_sql_query(query_productos, conn)
print("💼 Análisis de productos financieros:")
print(analisis_productos.to_string(index=False))

## 🔍 **5. HAVING - Filtros en Grupos Agregados**

### **¿Cuál es la diferencia entre WHERE y HAVING?**

- 🔹 **WHERE**: Filtra **antes** de agrupar (filas individuales)
- 🔹 **HAVING**: Filtra **después** de agrupar (grupos agregados)

**Sintaxis:**
```sql
SELECT columna, FUNCION_AGREGACION()
FROM tabla
WHERE condicion_individual
GROUP BY columna
HAVING condicion_grupo;
```

In [ ]:
# HAVING básico - Filtrar grupos
print("🔍 HAVING BÁSICO - Filtros en grupos")
print("=" * 50)

# Segmentos con más de 100 clientes
query_having = """
SELECT 
    segmento_cliente,
    COUNT(*) as cantidad_clientes,
    AVG(ingresos_mensuales) as ingreso_promedio
FROM clientes
WHERE estado = 'ACTIVO'
GROUP BY segmento_cliente
HAVING COUNT(*) > 100
ORDER BY cantidad_clientes DESC;
"""
segmentos_grandes = pd.read_sql_query(query_having, conn)
print("👥 Segmentos con más de 100 clientes activos:")
print(segmentos_grandes.to_string(index=False))

In [ ]:
# HAVING avanzado - Múltiples condiciones
print("🔍 HAVING AVANZADO - Condiciones múltiples")
print("=" * 50)

# Productos con alto volumen y saldo promedio
query_having_avanzado = """
SELECT 
    pf.categoria,
    COUNT(c.cuenta_id) as numero_cuentas,
    SUM(c.saldo_actual) as saldo_total,
    AVG(c.saldo_actual) as saldo_promedio
FROM productos_financieros pf
JOIN cuentas c ON pf.producto_id = c.producto_id
WHERE c.estado = 'ACTIVA'
GROUP BY pf.categoria
HAVING COUNT(c.cuenta_id) > 50 
   AND AVG(c.saldo_actual) > 1000000
ORDER BY saldo_total DESC;
"""
productos_estrategicos = pd.read_sql_query(query_having_avanzado, conn)
print("💼 Productos estratégicos (>50 cuentas y saldo prom >$1M):")
print(productos_estrategicos.to_string(index=False))

In [ ]:
# Comparación WHERE vs HAVING
print("⚖️ COMPARACIÓN: WHERE vs HAVING")
print("=" * 50)

# Ejemplo 1: WHERE (filtra antes de agrupar)
query_where = """
SELECT 
    segmento_cliente,
    COUNT(*) as clientes_altos_ingresos
FROM clientes
WHERE ingresos_mensuales > 3000000  -- Filtro ANTES de agrupar
GROUP BY segmento_cliente
ORDER BY clientes_altos_ingresos DESC;
"""
resultado_where = pd.read_sql_query(query_where, conn)
print("🔹 WHERE: Clientes con ingresos >$3M por segmento")
print(resultado_where.to_string(index=False))

# Ejemplo 2: HAVING (filtra después de agrupar)
query_having_comp = """
SELECT 
    segmento_cliente,
    COUNT(*) as total_clientes,
    AVG(ingresos_mensuales) as ingreso_promedio
FROM clientes
GROUP BY segmento_cliente
HAVING AVG(ingresos_mensuales) > 3000000  -- Filtro DESPUÉS de agrupar
ORDER BY ingreso_promedio DESC;
"""
resultado_having = pd.read_sql_query(query_having_comp, conn)
print("\n🔹 HAVING: Segmentos con ingreso promedio >$3M")
print(resultado_having.to_string(index=False))

## 💼 **6. KPIs Bancarios Fundamentales**

### **Key Performance Indicators en Banking**

Los KPIs bancarios son métricas esenciales para:
- 📊 **Monitoreo de performance**
- 🎯 **Toma de decisiones**
- 📈 **Análisis de rentabilidad**
- 🔍 **Identificación de oportunidades**
- ⚖️ **Cumplimiento regulatorio**

In [ ]:
# KPI 1: Concentración de Cartera
print("📊 KPI 1: CONCENTRACIÓN DE CARTERA")
print("=" * 50)

query_concentracion = """
SELECT 
    pf.categoria,
    COUNT(c.cuenta_id) as numero_cuentas,
    SUM(c.saldo_actual) as saldo_total,
    ROUND(SUM(c.saldo_actual) * 100.0 / 
          (SELECT SUM(saldo_actual) FROM cuentas), 2) as porcentaje_cartera,
    AVG(c.saldo_actual) as saldo_promedio
FROM productos_financieros pf
JOIN cuentas c ON pf.producto_id = c.producto_id
WHERE c.estado = 'ACTIVA'
GROUP BY pf.categoria
ORDER BY saldo_total DESC;
"""
concentracion = pd.read_sql_query(query_concentracion, conn)
print("💰 Concentración de cartera por producto:")
print(concentracion.to_string(index=False))

# Visualización
plt.figure(figsize=(12, 6))
plt.bar(concentracion['categoria'], concentracion['porcentaje_cartera'])
plt.title('Concentración de Cartera por Categoría de Producto')
plt.xlabel('Categoría')
plt.ylabel('Porcentaje de Cartera (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# KPI 2: Análisis de Segmentación de Clientes
print("👥 KPI 2: SEGMENTACIÓN DE CLIENTES")
print("=" * 50)

query_segmentacion_avanzada = """
SELECT 
    cl.segmento_cliente,
    COUNT(DISTINCT cl.cliente_id) as total_clientes,
    COUNT(DISTINCT c.cuenta_id) as total_cuentas,
    ROUND(COUNT(DISTINCT c.cuenta_id) * 1.0 / 
          COUNT(DISTINCT cl.cliente_id), 2) as cuentas_por_cliente,
    SUM(c.saldo_actual) as saldo_total_segmento,
    AVG(c.saldo_actual) as saldo_promedio_cuenta,
    AVG(cl.ingresos_mensuales) as ingreso_promedio_cliente
FROM clientes cl
LEFT JOIN cuentas c ON cl.cliente_id = c.cliente_id AND c.estado = 'ACTIVA'
WHERE cl.estado = 'ACTIVO'
GROUP BY cl.segmento_cliente
ORDER BY saldo_total_segmento DESC;
"""
segmentacion_avanzada = pd.read_sql_query(query_segmentacion_avanzada, conn)
print("📈 Análisis integral por segmento:")
print(segmentacion_avanzada.to_string(index=False))

In [ ]:
# KPI 3: Top Clientes (Pareto 80/20)
print("🏆 KPI 3: ANÁLISIS PARETO - TOP CLIENTES")
print("=" * 50)

query_top_clientes = """
WITH clientes_saldos AS (
    SELECT 
        cl.cliente_id,
        cl.nombre || ' ' || cl.apellidos as nombre_completo,
        cl.segmento_cliente,
        SUM(c.saldo_actual) as saldo_total_cliente,
        COUNT(c.cuenta_id) as numero_cuentas
    FROM clientes cl
    JOIN cuentas c ON cl.cliente_id = c.cliente_id
    WHERE cl.estado = 'ACTIVO' AND c.estado = 'ACTIVA'
    GROUP BY cl.cliente_id, cl.nombre, cl.apellidos, cl.segmento_cliente
)
SELECT 
    nombre_completo,
    segmento_cliente,
    numero_cuentas,
    saldo_total_cliente,
    ROUND(saldo_total_cliente * 100.0 / 
          (SELECT SUM(saldo_actual) FROM cuentas WHERE estado = 'ACTIVA'), 2) as porcentaje_cartera
FROM clientes_saldos
ORDER BY saldo_total_cliente DESC
LIMIT 20;
"""
top_clientes = pd.read_sql_query(query_top_clientes, conn)
print("🥇 Top 20 clientes por saldo total:")
print(top_clientes.to_string(index=False))

In [ ]:
# KPI 4: Distribución de Saldos
print("📊 KPI 4: DISTRIBUCIÓN DE SALDOS")
print("=" * 50)

query_distribucion = """
SELECT 
    CASE 
        WHEN saldo_actual <= 100000 THEN '1. Hasta $100K'
        WHEN saldo_actual <= 500000 THEN '2. $100K - $500K'
        WHEN saldo_actual <= 1000000 THEN '3. $500K - $1M'
        WHEN saldo_actual <= 5000000 THEN '4. $1M - $5M'
        WHEN saldo_actual <= 10000000 THEN '5. $5M - $10M'
        ELSE '6. Más de $10M'
    END as rango_saldo,
    COUNT(*) as numero_cuentas,
    SUM(saldo_actual) as saldo_total_rango,
    AVG(saldo_actual) as saldo_promedio,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM cuentas WHERE estado = 'ACTIVA'), 2) as porcentaje_cuentas,
    ROUND(SUM(saldo_actual) * 100.0 / (SELECT SUM(saldo_actual) FROM cuentas WHERE estado = 'ACTIVA'), 2) as porcentaje_saldos
FROM cuentas
WHERE estado = 'ACTIVA'
GROUP BY 
    CASE 
        WHEN saldo_actual <= 100000 THEN '1. Hasta $100K'
        WHEN saldo_actual <= 500000 THEN '2. $100K - $500K'
        WHEN saldo_actual <= 1000000 THEN '3. $500K - $1M'
        WHEN saldo_actual <= 5000000 THEN '4. $1M - $5M'
        WHEN saldo_actual <= 10000000 THEN '5. $5M - $10M'
        ELSE '6. Más de $10M'
    END
ORDER BY rango_saldo;
"""
distribucion_saldos = pd.read_sql_query(query_distribucion, conn)
print("📈 Distribución de cuentas por rango de saldo:")
print(distribucion_saldos.to_string(index=False))

# Gráfico de distribución
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Número de cuentas
ax1.bar(range(len(distribucion_saldos)), distribucion_saldos['numero_cuentas'])
ax1.set_title('Número de Cuentas por Rango')
ax1.set_xlabel('Rango de Saldo')
ax1.set_ylabel('Número de Cuentas')
ax1.set_xticks(range(len(distribucion_saldos)))
ax1.set_xticklabels(distribucion_saldos['rango_saldo'], rotation=45)

# Gráfico 2: Saldo total
ax2.bar(range(len(distribucion_saldos)), distribucion_saldos['saldo_total_rango'])
ax2.set_title('Saldo Total por Rango')
ax2.set_xlabel('Rango de Saldo')
ax2.set_ylabel('Saldo Total')
ax2.set_xticks(range(len(distribucion_saldos)))
ax2.set_xticklabels(distribucion_saldos['rango_saldo'], rotation=45)

plt.tight_layout()
plt.show()

## 📋 **7. Ejercicios Prácticos**

### **🎯 Retos para dominar las agregaciones**

In [ ]:
# EJERCICIO 1: Análisis de productos por rentabilidad
print("🎯 EJERCICIO 1: ANÁLISIS DE RENTABILIDAD")
print("=" * 50)
print("Objetivo: Identificar los productos más rentables del banco")
print()

# Tu turno: Completa esta consulta
query_ejercicio1 = """
-- EJERCICIO: Calcula por cada producto financiero:
-- 1. Número total de cuentas
-- 2. Saldo total
-- 3. Saldo promedio
-- 4. Tasa de interés promedio
-- 5. Solo productos con más de 10 cuentas
-- 6. Ordenado por saldo total descendente

SELECT 
    pf.nombre_producto,
    pf.categoria,
    -- COMPLETA AQUÍ:
    COUNT(c.cuenta_id) as total_cuentas,
    SUM(c.saldo_actual) as saldo_total,
    AVG(c.saldo_actual) as saldo_promedio,
    AVG(pf.tasa_interes_max) as tasa_promedio
FROM productos_financieros pf
JOIN cuentas c ON pf.producto_id = c.producto_id
WHERE c.estado = 'ACTIVA'
GROUP BY pf.producto_id, pf.nombre_producto, pf.categoria
HAVING COUNT(c.cuenta_id) > 10
ORDER BY saldo_total DESC;
"""

resultado_ejercicio1 = pd.read_sql_query(query_ejercicio1, conn)
print("💼 Productos más rentables:")
print(resultado_ejercicio1.to_string(index=False))

In [ ]:
# EJERCICIO 2: Análisis de comportamiento por fecha
print("🎯 EJERCICIO 2: ANÁLISIS TEMPORAL")
print("=" * 50)
print("Objetivo: Analizar patrones de apertura de cuentas")
print()

# Análisis por mes y año
query_ejercicio2 = """
SELECT 
    CAST(strftime('%Y', fecha_apertura) AS INTEGER) as año,
    CAST(strftime('%m', fecha_apertura) AS INTEGER) as mes,
    COUNT(*) as cuentas_abiertas,
    SUM(saldo_actual) as saldo_inicial_total,
    AVG(saldo_actual) as saldo_inicial_promedio
FROM cuentas
WHERE fecha_apertura IS NOT NULL
GROUP BY año, mes
HAVING año >= 2020  -- Solo años recientes
ORDER BY año DESC, mes DESC
LIMIT 24;  -- Últimos 24 meses
"""

comportamiento_temporal = pd.read_sql_query(query_ejercicio2, conn)
print("📅 Comportamiento de apertura por mes:")
print(comportamiento_temporal.to_string(index=False))

# Crear gráfico de tendencia
if len(comportamiento_temporal) > 0:
    comportamiento_temporal['periodo'] = comportamiento_temporal['año'].astype(str) + '-' + comportamiento_temporal['mes'].astype(str).str.zfill(2)
    
    plt.figure(figsize=(15, 6))
    plt.plot(comportamiento_temporal['periodo'], comportamiento_temporal['cuentas_abiertas'], marker='o')
    plt.title('Tendencia de Apertura de Cuentas')
    plt.xlabel('Período (Año-Mes)')
    plt.ylabel('Cuentas Abiertas')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# EJERCICIO 3: Dashboard de Métricas Clave
print("🎯 EJERCICIO 3: DASHBOARD DE MÉTRICAS")
print("=" * 50)
print("Objetivo: Crear un resumen ejecutivo del banco")
print()

# Métricas globales
def calcular_metricas_globales():
    """
    Calcula las métricas más importantes del banco
    """
    metricas = {}
    
    # Total de clientes activos
    query_clientes = "SELECT COUNT(*) as total FROM clientes WHERE estado = 'ACTIVO';"
    metricas['clientes_activos'] = pd.read_sql_query(query_clientes, conn)['total'].iloc[0]
    
    # Total de cuentas activas
    query_cuentas = "SELECT COUNT(*) as total FROM cuentas WHERE estado = 'ACTIVA';"
    metricas['cuentas_activas'] = pd.read_sql_query(query_cuentas, conn)['total'].iloc[0]
    
    # Saldo total
    query_saldo = "SELECT SUM(saldo_actual) as total FROM cuentas WHERE estado = 'ACTIVA';"
    metricas['saldo_total'] = pd.read_sql_query(query_saldo, conn)['total'].iloc[0]
    
    # Saldo promedio por cuenta
    query_promedio = "SELECT AVG(saldo_actual) as promedio FROM cuentas WHERE estado = 'ACTIVA';"
    metricas['saldo_promedio'] = pd.read_sql_query(query_promedio, conn)['promedio'].iloc[0]
    
    # Número de productos
    query_productos = "SELECT COUNT(*) as total FROM productos_financieros;"
    metricas['productos_disponibles'] = pd.read_sql_query(query_productos, conn)['total'].iloc[0]
    
    return metricas

# Ejecutar cálculo de métricas
metricas = calcular_metricas_globales()

print("📊 DASHBOARD EJECUTIVO - MÉTRICAS CLAVE")
print("=" * 60)
print(f"👥 Clientes Activos: {metricas['clientes_activos']:,}")
print(f"🏦 Cuentas Activas: {metricas['cuentas_activas']:,}")
print(f"💰 Saldo Total: ${metricas['saldo_total']:,.2f}")
print(f"📊 Saldo Promedio: ${metricas['saldo_promedio']:,.2f}")
print(f"📦 Productos Disponibles: {metricas['productos_disponibles']}")
print(f"🔢 Cuentas por Cliente: {metricas['cuentas_activas'] / metricas['clientes_activos']:.2f}")
print("=" * 60)

## 🎓 **8. Resumen y Próximos Pasos**

### **🏆 ¡Felicidades! Has completado el Módulo 2**

#### **✅ Conceptos Dominados:**
- 🧮 **Funciones de agregación**: COUNT, SUM, AVG, MAX, MIN
- 📊 **GROUP BY**: Agrupación simple y múltiple
- 🔗 **GROUP BY + JOIN**: Análisis integral de datos
- 🔍 **HAVING**: Filtros en grupos agregados
- 💼 **KPIs bancarios**: Métricas fundamentales
- 📈 **Análisis de distribución**: Segmentación y concentración

#### **💪 Habilidades Desarrolladas:**
- Calcular métricas de negocio bancario
- Analizar concentración de cartera
- Segmentar clientes por comportamiento
- Identificar patrones temporales
- Crear dashboards de métricas clave

---

### **🚀 Próximo Paso: Módulo 3 - JOINs y Relaciones**

#### **Lo que aprenderás:**
- 🔗 **INNER JOIN**: Relaciones exactas
- ↔️ **LEFT/RIGHT JOIN**: Inclusión de datos faltantes
- 🔄 **FULL OUTER JOIN**: Combinación completa
- 🏦 **Casos bancarios**: Análisis integral de cartera
- 📊 **Reportes complejos**: Múltiples fuentes de datos

#### **Aplicaciones bancarias:**
- Análisis cliente-cuenta-producto
- Historiales de transacciones
- Reportes regulatorios
- Análisis de riesgo crediticio

---

### **📚 Recursos de Apoyo**
- 📖 Consulta `GUIA_PRACTICA_SQL_AVANZADO.md` para ejercicios paso a paso
- 🔬 Usa `practica_sql_avanzada.py` para practicar con scripts
- 💪 Resuelve retos adicionales en `EJERCICIOS_PRACTICOS.md`
- 📊 Experimenta con nuevos KPIs usando tus propias ideas

**¡Estás listo para análisis más complejos! 🎯**

In [ ]:
# Cerrar conexión a la base de datos
if conn:
    conn.close()
    print("✅ Conexión a la base de datos cerrada")
    print("🎉 ¡Módulo 2 completado exitosamente!")
    print("📚 Continúa con el Módulo 3: JOINs y Relaciones")